
Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# Popular Sequence Dataset

In [19]:
import sys, os, functools, torch
sys.path.append('/home/jupyter/')
import numpy as np
import pandas as pd
from pathlib import Path

from paraMem.utils import modelHandlers, helpers

In [17]:
def load_pop_seqs(exclude_types:list=None):
    df = pd.read_csv("/home/jupyter/paraMem/data/popular_seqs/popular_seqs.csv")
    if exclude_types is not None:
        df = df[~df['type'].isin(exclude_types)] 
    seqs_str = df["paragraphs"].to_list()
    return seqs_str
    
seqs_str = load_pop_seqs(exclude_types=["idiom"])

In [22]:
torch.set_grad_enabled(False)
model = modelHandlers.load_model(model_type="gpt-neo-125M", DEVICE="cpu")
modelHandlers.set_no_grad(model, ["embed", "pos_embed", "unembed"])

Loaded pretrained model gpt-neo-125M into HookedTransformer
set no_grad on ['embed', 'pos_embed', 'unembed']


In [35]:
toks_NI = model.to_tokens(seqs_str)[:,:100] 
logs_NIT = model(toks_NI.detach())
nll_NIT = helpers.NegLogLik(logs_NIT)
nll_NI = helpers.gather_token_scores(nll_NIT, toks_NI, pool=[])

mean_nll = nll_NI.mean(-1) ## (1) NLL mean
nll_Nk, idcs_Nk = torch.topk(nll_NI, k=int(0.2*nll_NI.shape[-1]), largest=True, dim=-1) ## (2) minK
minK_nll = nll_Nk.mean(-1)


In [39]:
minK_nll

tensor([9.7275, 7.2896, 8.9266, 7.6466, 7.6885, 8.9015, 8.8846, 9.4874, 8.2477,
        8.8526, 9.2673, 7.2373])